In [6]:
import urllib
import requests
import bs4

# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from fake_useragent import UserAgent

# 1. 댓글 수집하기

In [7]:
url = 'http://corners.auction.co.kr/AllKill/AllDay.aspx'

## 1. urllib을 이용한 html page 가져오기

In [8]:
file, http = urllib.request.urlretrieve(url, './data/auction_list.html')

- 이미지, 영상 등의 파일을 바로 저장이 가능

In [9]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [10]:
html = response.read()

## 2. requests을 이용한 html page 가져오기

In [11]:
response = requests.get(url)

In [ ]:
# response.text
response.content

## 3. 리눅스에서 selenium 설정

### 1. chrome 설치
```
#> wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
#> sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
#> apt-get update
#> apt-get upgrade 
#> apt-get install google-chrome-stable
#> google-chrome --version
```

### 2. webdriver 다운로드
- url: https://chromedriver.chromium.org/downloads

```
#> wget https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_linux64.zip
#> unzip chromedriver_linux64.zip
```

## 4. selenium을 통한 html page 가져오기

In [14]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('./data/chromedriver', options=options)

In [15]:
browser.get(url)
response = browser.page_source

browser.quit()

## 5. BeautifulSoup을 이용한 html 파싱

In [21]:
# html 코드를 분석에 적당한 beautifulSoup 형태로 변환
html = bs4.BeautifulSoup( response, 'html.parser')

In [25]:
html.find('p')

<p class="layer-complainprocess__description"><strong>옥션은 거래 과정에서 발생하는 판매자와의 분쟁</strong> 또는 <strong>옥션이 제공하는 서비스에 대한 불만을 신속, 공정하게 해결</strong>하기 위해 고객님의 목소리에 귀를 기울이고 분쟁해결을 위해 최선을 다하겠습니다.</p>

In [28]:
condition = {
    'id':'coreSitemap',
    'class': 'core_sitemap'
}
html.find_all('div', attrs = condition)

[<div class="core_sitemap" id="coreSitemap" style="position: absolute; top: 0px; left: 40px; z-index: 1000002; display: none;"></div>]

In [32]:
element = html.select_one('ul#ItemList li a')

In [34]:
element['href']

'http://through.auction.co.kr/common/SafeRedirect.aspx?cc=178F&next=http%3a%2f%2fitempage3.auction.co.kr%2fDetailView.aspx%3fitemno%3dB737016790'

In [53]:
elements = html.select('ul#ItemList > li > div.inner > a')

In [57]:
item_list = []
for element in elements:
    if not 'javascript' in element['href']:
        # print(element['href'])
        item_list.append(element['href'])

## 6. 댓글 가져오기

### urllib을 통해서 가져오기

In [80]:
for url in item_list:
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    data = response.read().decode('euc-kr')
    html = bs4.BeautifulSoup(data, 'html.parser')
    element = html.select('div#VIPReview div#sectionVipReview div#divVipReview')
    print(element)
    break

[<div id="divVipReview"></div>]


### requests를 통한 댓글 가져오기

In [81]:
for url in item_list:
    response = requests.get(url)
    html = bs4.BeautifulSoup(response.text, 'html.parser')
    element = html.select('div#VIPReview div#sectionVipReview div#divVipReview')
    print(element)
    break

[<div id="divVipReview"></div>]


### selenium을 통한 댓글 가져오기

In [83]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
for url in item_list:
    try:
        browser = webdriver.Chrome('./data/chromedriver', options=options)
        browser.get(url)
        response = browser.page_source

        html = bs4.BeautifulSoup(response, 'html.parser')
        element = html.select('div#VIPReview div#sectionVipReview div#divVipReview')
        print(element)
        browser.quit()
    except:
        browser.quit()
        break
    break